In [27]:
import pandas as pd
import numpy as np
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
import enchant
from enchant.checker import SpellChecker
import enchant

In [3]:
import sys
print("Python version")
print (sys.version)

Python version
3.7.7 (tags/v3.7.7:d7c567b08f, Mar 10 2020, 10:41:24) [MSC v.1900 64 bit (AMD64)]


In [4]:
df = pd.read_csv('../data/raw_data.csv')

# Making the text more uniform

Remove urls

In [5]:
df.body = df.body.replace(r'http\S+', '', regex=True)

Change acronyms to words for uniformity. 
* https://www.netlingo.com/acronyms.php
* https://blog.adioma.com/internet-acronyms-intro-list-infographic/

In [6]:
# internet slangs
acronyms = {
    'btw': 'by the way',
    'afaik': 'as far as i know',
    'afk': 'away from keyboard',
    'asap': 'as soon as possible',
    'atm': 'at the moment',
    'bbl': 'be back later',
    'brb': 'be right back',
    'b4': 'before',
    'cya': 'see ya',
    'dae': 'does anyone else',
    'eli5': 'explain like i\'m five',
    'fyi': 'for your information',
    'faq': 'frequently asked questions',
    'gg': 'good game',
    'gr8': 'great',
    'imo': 'in my opinion',
    'imho': 'in my honest opinion',
    'iirc': 'if i remember correctly',
    'irl': 'in real life',
    'l8r': 'later',
    'm8': 'mate',
    'mrw': 'my reaction when',
    'mfw': 'my face when',
    'pita': 'pain in the ass',
    'thx': 'thanks',
    'til': 'today i learned',
    'tbh': 'to be honest',
    'u': 'you',
    'y': 'why',
    'r': 'are',
    'ur': 'your',
    'w8': 'wait',
    '2nite': 'tonight',
}

In [7]:
# spell checker with uppercase some letters later on
df.body = df.body.str.lower()
# the ’ was causing issues, took a while to notice
df.body = df.body.str.replace('’', '\'')

In [8]:
df.body = df.body.apply(lambda x: ' '.join(acronyms.get(word, word) for word in x.split()))

Fix as many spelling errors as possible

In [9]:
spell_checker = SpellChecker("en_UK","en_US")

In [11]:
def correct_error(body):
    spell_checker.set_text(body)
    for err in spell_checker:
        if len(err.suggest())>0: 
            sug = err.suggest()[0]
            err.replace(sug)
    return spell_checker.get_text()

In [12]:
df.body = df.body.apply(lambda row: correct_error(row))

Remove stop words and replace negations with "not"

In [9]:
# again, thanks to spell checker
df.body = df.body.str.lower()

In [10]:
stopwords_list = stopwords.words('english')
negations = ['don\'t', 'aint' 'aren\'t', 'couldn\'t','didn\'t', 
             'doesn\'t', 'hadn\'t', 'hasn\'t', 'haven\'t', 'isn\'t', 
             'mightn\'t', 'mustn\'t', 'needn\'t', 'shouldn\'t', 'wasn\'t', 
             'weren\'t', 'won\'t', 'wouldn\'t', 'nor', 'not']
stopwords_list = [el for el in stopwords_list if el not in negations]
missing_words = ['i\'m', 'i\'d']
stopwords_list.extend(missing_words)

In [11]:
regx = r'\b(?:{})\b'.format('|'.join(negations))
df.body = df.body.str.replace(regx, 'not')

In [12]:
df.body = df.body.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [13]:
df.head(10)

,body,positive,negative,neutral,rated,comment_id,video_id,date
0,"*stretched past 10 minute mark ad revenue, see!*",0,1,0,1,UgztP4lVR-Epv5HlSXN4AaABAg,ItYOdWRo0JY,2020-01-10T20:24:33Z
1,big time scam,0,1,0,1,UgzZubyLG5FtZu7qlal4AaABAg,ItYOdWRo0JY,2019-08-01T15:45:49Z
2,recycle face,0,1,0,1,UgzSMwb88ntjkYHQYaN4AaABAg,ItYOdWRo0JY,2019-06-19T04:00:06Z
3,god dang twat,0,1,0,1,UgwS5xQLLnyIzNUS4bp4AaABAg,ItYOdWRo0JY,2019-06-16T00:49:53Z
4,not give away one 3 fans recognized you?,0,1,0,1,Ugz1EJ6K2F0CYUX5NrN4AaABAg,ItYOdWRo0JY,2019-03-31T00:50:58Z
5,trash trash stupid unsubscribing,0,1,0,1,UgyI5hLjBfIN99uVWDt4AaABAg,ItYOdWRo0JY,2019-03-16T03:50:36Z
6,crazy man kkk,0,1,0,1,UgwDghsKi_E8H3xYe-V4AaABAg,ItYOdWRo0JY,2019-02-22T17:48:07Z
7,physically hate,0,1,0,1,Ugxo2RWpZiVb6i0dGO54AaABAg,ItYOdWRo0JY,2019-02-04T15:31:57Z
8,first video see already fake. good job,0,1,0,1,UgwkHi8FfFqozu7c4KJ4AaABAg,ItYOdWRo0JY,2019-02-02T09:09:34Z
9,messed,0,1,0,1,Ugy-UnErAg23Btoph3x4AaABAg,ItYOdWRo0JY,2019-02-01T01:53:55Z


List of emojis defined from https://en.wikipedia.org/wiki/List_of_emoticons , https://emojipedia.org/people/

Other references:
* https://www.urbandictionary.com/define.php?term=%F0%9F%92%80
* https://www.urbandictionary.com/define.php?term=%F0%9F%94%A5

In [16]:
import emoji
# list of tagged emoticons from above links
from smileys import smileys
from smileys import sent_acronyms
# for removing untagged emoji
import demoji
#demoji.download_codes()

In [17]:
def replace_repeating_emoji(text):
    uniques = set()
    final_string = list()
    text_arr = [item for item in emoji.get_emoji_regexp().split(text) if not item == '']
    for e in text_arr:
        if not bool(emoji.get_emoji_regexp().search(e)):
            final_string.append(smileys.get(e, e))
        else:
            if e not in uniques:
                uniques.add(e)
                final_string.append(smileys.get(e, e))
    return ' '.join(final_string)

In [18]:
# find repeating emoticons and remove repetitions, and tag emoticons
df.body = df.body.apply(lambda x: replace_repeating_emoji(x) if (bool(emoji.get_emoji_regexp().search(x)) and bool(re.search(r'(.)\1', x))) else x)

In [19]:
# remove untagged emoticons
df.body = df.body.apply(lambda x : demoji.replace(x, ''))

Set sentiment on acronyms (such as 'lol')

In [20]:
df.body = df.body.apply(lambda x: ' '.join(sent_acronyms.get(word, word) for word in x.split()))

Remove hashtags

In [31]:
df.body = df.body.apply(lambda x: ' '.join([word for word in x.split() if '#' not in word]))

In [32]:
df.loc[df.body.str.contains('#')]

,body,positive,negative,neutral,rated,comment_id,video_id,date


Remove punctuation

In [33]:
import string
df.body = df.body.str.replace('[{}]'.format(string.punctuation), '')

In [34]:
df.tail(10)

,body,positive,negative,neutral,rated,comment_id,video_id,date
1323,many people believe,0,1,0,1,UgiH2C_pE_zFgHgCoAEC,cRJjXCK6H6Y,2016-08-21T12:48:52Z
1324,cant believe d sorry currently not access feat...,0,1,0,1,Ugxl39L-FybtONur09J4AaABAg,8d_202l55LU,2017-12-16T05:41:20Z
1325,honestly one better videos ive seen days gave ...,1,0,0,1,UgyyZgg2kULZsLwTMjF4AaABAg,YbJOTdZBX1g,2021-02-01T19:21:37Z
1326,feel stabbed back leafy not finish like hoped,0,1,0,1,UgiveoyqvwtYdXgCoAEC,Rqy_5PmID6k,2016-06-22T20:45:17Z
1327,mom,0,1,0,1,UgyTfLvliRAjqdG83jd4AaABAg,udBgnu1hbLU,2020-01-03T22:09:58Z
1328,oh bad boo fake news station,0,1,0,1,UgyyBBhr7F04uU5TkKh4AaABAg,FmE8DwZ6RHk,2018-02-01T08:07:41Z
1329,sound racist pewds fuckin leech,0,1,0,1,UgwDItRoRWNWE3Me4ZF4AaABAg,_UZbgH3pOBA,2017-11-11T03:37:51Z
1330,donald right shut cnn,0,1,0,1,UgxY_W2PQVusRKHAuPl4AaABAg,FmE8DwZ6RHk,2018-01-28T15:05:38Z
1331,im sorry cant take care dog want baby please,0,1,0,1,UgwZBfZlFUxofuQpKBB4AaABAg,HVjlcUtuENM,2018-05-03T10:45:50Z
1332,biased fuck,0,1,0,1,UgjnBid2wZzZn3gCoAEC,cRJjXCK6H6Y,2016-09-03T13:05:10Z


Remove repeating vowels and consonants

In [35]:
# https://stackoverflow.com/questions/46701245/how-to-replace-multiple-consecutive-repeating-characters-into-1-character-in-pyt
df.body = df.body.apply(lambda x: ' '.join([re.sub(r'[^\w\s]|(.)(?=\1)', '', word) for word in x.split()]))

Tag any sequence of "ha" or "ah" (for example, "ahaha" or "haha") as a "laugh"

In [48]:
df.body = df.body.apply(lambda x: ' '.join([re.sub(r'([ha]+[ah]+).*\1', r'laugh', word) for word in x.split()]))

To fix:
* if spell checking is not performed, make sure that for negations misspelled words are considered, like 'cant' and so on.

To do:
* stemming
* consider replacing insults into a tag, like "bad_word"
* tokenize

after that
* tf-idf for feature extraction? look into dictionaries: hownet, ntusd
* word2vec for embedding